<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U13/ANN13_neural_style_transfer_TEMPLATE_pl.ipynb)

# Neural Style Transfer
**Neural Style Transfer (NST)** ist ein Verfahren im Bereich des Deep Learning, das es ermöglicht, den **Inhalt eines Bildes** mit dem **künstlerischen Stil eines anderen Bildes** zu kombinieren. Ziel ist es, ein neues Bild zu erzeugen, das die semantische Struktur des einen Bildes (z. B. eine Landschaft) mit der künstlerischen Ästhetik eines anderen Bildes (z. B. ein Van-Gogh-Gemälde) vereint.
## Ziel

Ein neues Bild generieren, das:

- **Inhaltlich dem Content-Bild** entspricht  
- **Stilistisch dem Style-Bild** ähnelt
## Funktionsweise

NST verwendet ein **vortrainiertes Convolutional Neural Network (CNN)** – typischerweise **VGG19** – um Bilder in sogenannte **Feature-Repräsentationen** zu überführen. Dabei wird unterschieden zwischen:

- **Content-Features**: erfassen die räumliche Struktur und Anordnung von Objekten  
- **Style-Features**: erfassen die Textur, Farbverteilung und Muster – über sogenannte **Gram-Matrizen**

Das **generierte Bild** wird schrittweise angepasst, sodass es ähnliche Feature-Repräsentationen wie die Eingabebilder aufweist.


## Komponenten im Detail

| Komponente       | Beschreibung                                                                 |
|------------------|------------------------------------------------------------------------------|
| **Content Loss** | Misst die Abweichung der Content-Features zwischen generiertem und Content-Bild |
| **Style Loss**   | Misst die Abweichung der Gram-Matrizen zwischen generiertem und Style-Bild     |
| **TV Loss**      | (optional) Glättet das Bild, um visuelle Artefakte zu reduzieren               |
| **Optimierung**  | Das Bild selbst wird trainiert – nicht das Netzwerk!                          |


## Technischer Ablauf

1. **Initialisierung**: Starte mit einem Rauschbild oder einer Kopie des Content-Bildes  
2. **Feature Extraktion**: VGG-Netz extrahiert Features aus Content-, Style- und generiertem Bild  
3. **Loss-Berechnung**: Vergleiche die Features und berechne Content- und Style-Loss  
4. **Backpropagation**: Optimiere das Bild so, dass der kombinierte Loss minimiert wird  
5. **Visualisierung**: Das Bild entwickelt sich mit jeder Iteration in Richtung „stilisiertes Ergebnis“


## Anwendung in dieser Übung

In dieser Übung wirst du:

- Ein VGG19-Modell als Feature-Extractor nutzen  
- Content- und Style-Features aus Beispielbildern extrahieren  
- Ein neues Bild durch Optimierung erzeugen  
- Verlaufsplots zur Analyse der Verluste über die Iterationen erstellen

## Typische Beispiele

| Content-Bild                | Style-Bild                   | Ergebnis                              |
|-----------------------------|------------------------------|----------------------------------------|
| Landschaftsfoto             | Monet-Gemälde                | Landschaft im impressionistischen Stil |
| Porträtfoto                 | Picasso-Zeichnung            | Abstraktes Porträt                     |
| Stadtbild                   | Van Gogh (Sternennacht)      | Leuchtende, wirbelnde Stadtansicht     |

## (a) Setup der Umgebung

Die Funktion `setup_environment()` prüft, ob alle benötigten Dateien für den Neural Style Transfer vorhanden sind – insbesondere das vortrainierte VGG19-Modell und zwei Beispielbilder (Content & Style). Falls die Dateien nicht vorhanden sind, werden sie automatisch aus dem Internet heruntergeladen und an die richtigen Stellen gespeichert.

### Was genau passiert:

1. **VGG19-Modell laden:**
   - Prüft, ob die Datei `models/vgg19-d01eb7cb.pth` vorhanden ist
   - Wenn nicht, wird das vortrainierte VGG19-Modell von der Universität Michigan heruntergeladen und in den Ordner `models/` gespeichert

2. **Beispielbilder laden:**
   - Prüft, ob `images/1-content.png` und `images/1-style.jpg` existieren
   - Falls nicht:
     - Wird ein ZIP-Archiv von GitHub heruntergeladen (`master.zip`)
     - Es wird entpackt
     - Die beiden Beispielbilder werden aus dem entpackten Ordner in den Ordner `images/` verschoben

3. Am Ende wird bestätigt, dass das Setup erfolgreich abgeschlossen wurde

> Diese Funktion erleichtert den Einstieg und stellt sicher, dass das Notebook direkt lauffähig ist – ohne manuelles Herunterladen von Dateien.

In [ ]:
import os
import urllib.request
import zipfile


def setup_environment():
    # Basisverzeichnis
    base_dir = "data"
    model_dir = os.path.join(base_dir, "models")
    image_dir = os.path.join(base_dir, "images")

    # Check if VGG-Modell existiert
    vgg_path = os.path.join(model_dir, "vgg19-d01eb7cb.pth")
    if not os.path.exists(vgg_path):
        print("Lade VGG19-Gewichte herunter ...")
        os.makedirs(model_dir, exist_ok=True)
        url = "https://web.eecs.umich.edu/~justincj/models/vgg19-d01eb7cb.pth"
        urllib.request.urlretrieve(url, vgg_path)

    # Check ob Bilder existieren
    content_img = os.path.join(image_dir, "1-content.png")
    style_img = os.path.join(image_dir, "1-style.jpg")
    if not os.path.exists(content_img) or not os.path.exists(style_img):
        print("Lade Beispielbilder herunter ...")
        os.makedirs(image_dir, exist_ok=True)
        zip_path = os.path.join(base_dir, "master.zip")
        if not os.path.exists(zip_path):
            urllib.request.urlretrieve(
                "https://github.com/iamRusty/neural-style-pytorch/archive/master.zip",
                zip_path,
            )
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(base_dir)

        extracted_path = os.path.join(base_dir, "neural-style-pytorch-master", "images")
        os.rename(os.path.join(extracted_path, "1-content.png"), content_img)
        os.rename(os.path.join(extracted_path, "1-style.jpg"), style_img)

    print("✅ Setup abgeschlossen. Dateien sind im 'data/' Ordner gespeichert.")


# Setup die Umgebung
setup_environment()


#### Noch mehr Bilder um selber umzuschalten


In [ ]:
# Zielordner
target_dir = "Bilder/StyleTransfer"
os.makedirs(target_dir, exist_ok=True)

# Bildliste aus Screenshot (alle .jpg + .png außer readme.md)
filenames = [
    "Klimt2.jpg",
    "Kopenhagen.jpg",
    "Mosaic2.jpg",
    "Sonnenblumen.jpg",
    "Van_Gogh.jpg",
    "Vassily_Kandinsky7.jpg",
    "candy.jpg",
    "composition-with-figures_popova.jpg",
    "composition.jpg",
    "graffiti.jpg",
    "popova.png",
    "portrait.jpg",
    "portrait_mann.jpg",
    "portrait_women.jpg",
    "simpsons.jpg",
    "udnie.jpg",
]

# Basis-URL
base_url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/data/StyleTransfer/"

# Download-Schleife
for name in filenames:
    save_path = os.path.join(target_dir, name)
    if os.path.exists(save_path):
        print(f"✅ Bereits vorhanden: {name}")
        continue
    url = base_url + name
    try:
        urllib.request.urlretrieve(url, save_path)
        print(f"✅ Heruntergeladen: {name}")
    except Exception as e:
        print(f"❌ Fehler bei {name}: {e}")


In diesem Abschnitt werden alle benötigten Bibliotheken für den Neural Style Transfer geladen. Sie ermöglichen Bildverarbeitung, Modellhandling, Visualisierung und numerische Optimierung.

In [ ]:
# -----------------------------------------------
# Neural Style Transfer in PyTorch
# Ziel: Inhalt eines Bildes mit dem Stil eines anderen kombinieren
# -----------------------------------------------

import os
import cv2
import copy
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import plotly.subplots as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from torchvision import models, transforms


## (b) Hyperparameter und Einstellungen

In diesem Abschnitt werden alle zentralen Konfigurationsparameter für den Style Transfer definiert. Sie bestimmen, **wie stark Inhalt und Stil gewichtet werden**, wie das Bild initialisiert wird und welche Optimierungsmethode verwendet wird.

### Modell- und Optimierungsparameter

| Parameter         | Bedeutung                                                                 |
|-------------------|---------------------------------------------------------------------------|
| `MAX_IMAGE_SIZE`  | Maximale Kantenlänge der Eingabebilder (Skalierung)                       |
| `OPTIMIZER`       | Optimierer: `adam` (schnell, stabil) oder `lbfgs` (klassisch, präzise)    |
| `ADAM_LR`         | Lernrate für den Adam-Optimierer                                          |
| `CONTENT_WEIGHT`  | Gewichtung des Inhaltsverlustes (Content Loss)                            |
| `STYLE_WEIGHT`    | Gewichtung des Stilverlustes (Style Loss)                                 |
| `TV_WEIGHT`       | Gewichtung des Glättungsfaktors (Total Variation Loss)                    |
| `NUM_ITER`        | Anzahl der Optimierungsschritte                                           |
| `SHOW_ITER`       | Nach wie vielen Iterationen ein Zwischenbild angezeigt wird               |
| `INIT_IMAGE`      | Startbild: `"random"` oder `"content"`                                    |
| `PRESERVE_COLOR`  | Soll die Farbgebung des Content-Bildes erhalten bleiben? (`True/False`)   |
| `PIXEL_CLIP`      | Soll das Bild nach jeder Iteration auf gültige Pixelwerte begrenzt werden?|

### Pfade zu Ressourcen

| Variable       | Beschreibung                                 |
|----------------|----------------------------------------------|
| `CONTENT_PATH` | Pfad zum Content-Bild                        |
| `STYLE_PATH`   | Pfad zum Style-Bild                          |
| `VGG19_PATH`   | Pfad zu den vortrainierten VGG19-Gewichten   |
| `POOL`         | Art des Poolings im Netzwerk (`max` oder `avg`) |

### Gerät wählen

```python
device = "cuda" if torch.cuda.is_available() else "cpu"
```

> Diese Zeile prüft automatisch, ob eine GPU (CUDA) verfügbar ist, und nutzt sie – andernfalls wird die CPU verwendet.

In [ ]:
# -----------------------------------------------
# Hyperparameter & Einstellungen
# -----------------------------------------------
MAX_IMAGE_SIZE = 512
OPTIMIZER = "adam"  # 'adam' oder 'lbfgs'
ADAM_LR = 10  # Lernrate
CONTENT_WEIGHT = 5e0  # Gewichtung des Inhaltsverlustes
STYLE_WEIGHT = 1e2  # Gewichtung des Stilverlustes
TV_WEIGHT = 1e-3  # Gewichtung des Total Variation Loss
NUM_ITER = 500  # Anzahl der Optimierungsiterationen
SHOW_ITER = 100  # Anzeigeintervall
INIT_IMAGE = "random"  # 'random' oder 'content'
PRESERVE_COLOR = "True"  # Soll die Farbe des Content-Bildes beibehalten werden?
PIXEL_CLIP = "True"  # Pixel nach jeder Iteration clippen

# Pfade zu Bildern
# =========== CONTENT PATHS =========== Bitte nur jeweils einen Pfad aktivieren
CONTENT_PATH = "data/images/1-content.png"
# CONTENT_PATH = "Bilder/StyleTransfer/Kopenhagen.jpg"
# CONTENT_PATH = "Bilder/StyleTransfer/portrait_mann.jpg"
# CONTENT_PATH = "Bilder/StyleTransfer/portrait_women.jpg"
# CONTENT_PATH = "Bilder/StyleTransfer/portrait.jpg"
# CONTENT_PATH = "Bilder/StyleTransfer/simpsons.jpg"
# =========== STYLE PATHS =========== Bitte nur jeweils einen Pfad aktivieren
STYLE_PATH = "data/images/1-style.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/candy.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/Van_Gogh.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/udnie.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/Vassily_Kandinsky7.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/composition.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/composition-with-figures_popova.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/graffiti.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/Mosaic2.jpg"
# STYLE_PATH = "Bilder/StyleTransfer/Sonnenblumen.jpg"
# =========== VGG PATH ===========
VGG19_PATH = "data/models/vgg19-d01eb7cb.pth"
POOL = "max"

# Gerät festlegen
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Verwendetes Gerät:", device)


## (c) Hilfsfunktionen für die Bildverarbeitung

Diese Funktionen unterstützen das Laden, Anzeigen, Speichern und die Farbübertragung von Bildern, was zentral für die Visualisierung und Verarbeitung im Neural Style Transfer ist.

### Funktionsübersicht: Bildverarbeitung

| Funktion           | Zweck                                                  | Besonderheiten                                       |
|--------------------|---------------------------------------------------------|------------------------------------------------------|
| `load_image(path)` | Bild aus Datei laden (BGR-Format, OpenCV)              | Nutzt `cv2.imread()`                                 |
| `show(img)`        | Bild anzeigen (matplotlib, RGB-Konvertierung)          | Skaliert Pixel auf [0, 1], wandelt BGR → RGB         |
| `saveimg(img, iters)` | Stilisiertes Bild speichern                           | Erstellt Ordner `style_transfer_pictures/`, wandelt zu `uint8` |
| `transfer_color(src, dest)` | Farbübertragung vom Content-Bild auf das Stilbild | Arbeitet mit YCrCb-Farbraum, ersetzt Helligkeit      |

In [ ]:
# -----------------------------------------------
# Hilfsfunktionen für Bildverarbeitung
# -----------------------------------------------
def load_image(path):
    return cv2.imread(path)  # BGR Format


def show(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.array(img / 255).clip(0, 1)
    plt.figure(figsize=(10, 5))
    plt.imshow(img)
    plt.axis("off")
    plt.show()


def saveimg(img, iters):
    # Erstelle den Ordner 'style_transfer_pictures', falls er nicht existiert
    output_dir = "data/style_transfer_pictures"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if PIXEL_CLIP == "True":
        img = img.clip(0, 255)
    img_uint8 = img.astype(np.uint8)
    # Speichere das Bild im Ordner 'style_transfer_pictures'
    cv2.imwrite(os.path.join(output_dir, f"out{iters}.png"), img_uint8)


def transfer_color(src, dest):
    if PIXEL_CLIP == "True":
        src, dest = src.clip(0, 255), dest.clip(0, 255)
    H, W, _ = src.shape
    dest = cv2.resize(dest, dsize=(W, H), interpolation=cv2.INTER_CUBIC)
    dest_gray = cv2.cvtColor(dest, cv2.COLOR_BGR2GRAY)
    src_yiq = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)
    src_yiq[..., 0] = dest_gray
    return cv2.cvtColor(src_yiq, cv2.COLOR_YCrCb2BGR)


## (d) Tensor-Bild Konvertierung

Für das Training und die Optimierung im Neural Style Transfer müssen Bilder als **Tensors** verarbeitet werden. Diese beiden Funktionen übernehmen die Konvertierung zwischen **NumPy-Bilddaten (OpenCV)** und **PyTorch-Tensoren**, wobei sie auch das **Preprocessing bzw. Denormalisieren** mit den VGG19-spezifischen Mittelwerten übernehmen.


### Funktionen im Detail

#### `itot(img)`
**Image → Tensor**

- Wandelt ein Bild (`np.ndarray`, BGR) in einen PyTorch-Tensor um
- Skaliert das Bild auf die maximale Größe `MAX_IMAGE_SIZE`
- Normalisiert die Farbkanäle mit den VGG19-Trainingsmittelwerten:
  `[103.939, 116.779, 123.68]` (BGR-Reihenfolge)
- Multipliziert mit 255, da `ToTensor()` standardmäßig Werte in `[0, 1]` gibt
- Fügt Batch-Dimension `[1, C, H, W]` hinzu

#### `ttoi(tensor)`
**Tensor → Image**

- Entfernt die Batch-Dimension
- Revertiert die Normalisierung mit den negativen VGG-Mittelwerten
- Wandelt den Tensor wieder in ein NumPy-Bild `[H, W, C]` um (RGB)
- Gibt ein Bild-Array mit Werten im Bereich `[0, 255]` (float) zurück


### Funktionsübersicht

| Funktion     | Richtung         | Zweck                                         | Besonderheiten                               |
|--------------|------------------|-----------------------------------------------|----------------------------------------------|
| `itot(img)`  | Bild → Tensor     | Vorverarbeitung & Normalisierung für VGG19    | Skaliert Bild, wendet VGG-Mean-Norm an       |
| `ttoi(tensor)` | Tensor → Bild   | Rücktransformation zum Bildformat             | Macht Bild darstellbar, entfernt VGG-Norm    |


> Diese Funktionen sorgen dafür, dass deine Bilder korrekt als Netzwerk-Input verarbeitet werden können – und später auch wieder als visuell interpretierbare Ausgaben vorliegen.

In [ ]:
# -----------------------------------------------
# Tensor-Image Konvertierung
# -----------------------------------------------
def itot(img):
    H, W, _ = img.shape
    image_size = tuple([int((MAX_IMAGE_SIZE / max(H, W)) * x) for x in [H, W]])
    itot_t = transforms.Compose(
        [transforms.ToPILImage(), transforms.Resize(image_size), transforms.ToTensor()]
    )
    normalize_t = transforms.Normalize([103.939, 116.779, 123.68], [1, 1, 1])
    tensor = normalize_t(itot_t(img) * 255).unsqueeze(0)
    return tensor


def ttoi(tensor):
    ttoi_t = transforms.Compose(
        [transforms.Normalize([-103.939, -116.779, -123.68], [1, 1, 1])]
    )
    tensor = tensor.squeeze()
    img = ttoi_t(tensor).cpu().numpy().transpose(1, 2, 0)
    return img


## (e) Bilder vorbereiten

In diesem Schritt werden das **Content-Bild** und das **Style-Bild** geladen und visualisiert. Diese beiden Bilder sind die Grundlage für den späteren Stiltransfer.


In [ ]:
# -----------------------------------------------
# Bilder vorbereiten
# -----------------------------------------------
content_img = load_image(CONTENT_PATH)
style_img = load_image(STYLE_PATH)
print("Content Image Shape:", content_img.shape, "-------------")
show(content_img)
print("Style Image Shape:", style_img.shape, "-------------")
show(style_img)


## (f) VGG19 Feature-Extractor vorbereiten

In diesem Abschnitt wird ein **vortrainiertes VGG19-Netzwerk** als **Feature-Extractor** geladen und vorbereitet. Dieses CNN wird **nicht trainiert**, sondern lediglich verwendet, um **semantische Inhalte und stilistische Merkmale** aus den Bildern zu extrahieren.



### Was passiert hier?

1. **Laden des VGG19-Modells**
   ```python
   vgg = models.vgg19(pretrained=False)
   vgg.load_state_dict(torch.load(VGG19_PATH), strict=False)
   ```
   - Das Modell wird **nicht automatisch** von `torchvision` heruntergeladen (`pretrained=False`)
   - Stattdessen werden manuell geladene Gewichte verwendet (`VGG19_PATH`)
   - Es handelt sich um ein **auf ImageNet vortrainiertes Modell**

2. **Extraktion des Feature-Teils**
   ```python
   model = copy.deepcopy(vgg.features).to(device)
   ```
   - Nur der **Feature-Teil des VGG19-Netzes** wird verwendet
   - Der Klassifikationskopf (`classifier`) wird entfernt

3. **Deaktivieren der Gradientenberechnung**
   ```python
   for param in model.parameters():
       param.requires_grad = False
   ```
   - Das Modell bleibt **eingefroren** während des Style Transfers
   - Es dient nur zur **Berechnung der Verluste**, nicht zur Optimierung


### Warum VGG19?

- VGG19 ist ein tiefes CNN mit vielen kleinen Faltungen
- Seine Zwischenlayer repräsentieren sehr gut:
  - **Inhalt** (mittlere Layer)
  - **Stil** (frühe + tiefe Layer, über Gram-Matrizen)
- Durch das Einfrieren bleiben diese Repräsentationen stabil und zuverlässig


> Das Modell extrahiert wichtige Merkmale aus Content- und Style-Bild und ermöglicht die **Berechnung der entsprechenden Verluste**, die das generierte Bild formen.

In [ ]:
# -----------------------------------------------
# VGG19 Feature-Extractor vorbereiten
# -----------------------------------------------
vgg = models.vgg19(weights=None)
vgg.load_state_dict(torch.load(VGG19_PATH, weights_only=False), strict=False)
model = copy.deepcopy(vgg.features).to(device)

# Gradienten ausschalten
for param in model.parameters():
    param.requires_grad = False  # Parameter nicht trainierbar machen


## (g) Loss-Funktionen

Beim Neural Style Transfer basiert die Optimierung auf verschiedenen Verlustfunktionen, die sicherstellen, dass das generierte Bild sowohl inhaltlich dem Content-Bild als auch stilistisch dem Style-Bild ähnelt. Zusätzlich wird ein Glättungsterm verwendet, um visuelle Artefakte zu vermeiden.



### Übersicht der verwendeten Loss-Funktionen

| Funktion        | Zweck                                                         |
|-----------------|---------------------------------------------------------------|
| **Content Loss** | Misst Ähnlichkeit zum Content-Bild anhand von VGG-Features     |
| **Style Loss**   | Misst Ähnlichkeit zum Style-Bild über Gram-Matrizen           |
| **TV Loss**      | Total Variation Loss – sorgt für glatte Übergänge im Bild      |



### Definitionen im Code

#### Mean Squared Error Loss (Grundlage)
```python
mse_loss = nn.MSELoss()
```
Der mittlere quadratische Fehler bildet die Basis aller Verluste.



#### Gram-Matrix: Stil-Repräsentation
```python
def gram(tensor):
    B, C, H, W = tensor.shape
    x = tensor.view(C, H * W)
    return torch.mm(x, x.t())
```
- Die Gram-Matrix misst die **Korrelation zwischen Feature-Kanälen**
- Je ähnlicher zwei Kanäle feuern, desto höher ihr Wert
- Sie beschreibt **Textur und Stil**, unabhängig von der Position im Bild



#### Content Loss
```python
def content_loss(g, c):
    return mse_loss(g, c)
```
- Misst die Differenz der Features zwischen dem generierten Bild `g` und dem Content-Bild `c`
- Nutzt eine mittlere Schicht (z. B. `relu4_2`) aus dem VGG-Netz



#### Style Loss
```python
def style_loss(g, s):
    c1, _ = g.shape
    return mse_loss(g, s) / (c1**2)
```
- Nutzt Gram-Matrizen aus mehreren VGG-Schichten (z. B. `relu1_2`, `relu3_3`, ...)
- Die Division durch `c1²` (Kanalanzahl²) normalisiert die Verlustgröße



#### Total Variation Loss
```python
def tv_loss(c):
    x = c[:, :, 1:, :] - c[:, :, :-1, :]
    y = c[:, :, :, 1:] - c[:, :, :, :-1]
    return torch.sum(torch.abs(x)) + torch.sum(torch.abs(y))
```
- Belohnt **räumlich glatte Strukturen**
- Hilft, **visuelles Rauschen** im Bild zu vermeiden
- Optional, aber häufig mit großem Einfluss auf das Ergebnis



> Diese Verluste werden gewichtet kombiniert und bilden die Zielfunktion, nach der das generierte Bild optimiert wird.

In [ ]:
# -----------------------------------------------
# Loss-Funktionen
# -----------------------------------------------
mse_loss = nn.MSELoss()


def gram(tensor):
    B, C, H, W = tensor.shape
    x = tensor.view(C, H * W)
    return torch.mm(x, x.t())


def content_loss(g, c):
    return mse_loss(g, c)


def style_loss(g, s):
    c1, _ = g.shape
    return mse_loss(g, s) / (c1**2)


def tv_loss(c):
    x = c[:, :, 1:, :] - c[:, :, :-1, :]
    y = c[:, :, :, 1:] - c[:, :, :, :-1]
    return torch.sum(torch.abs(x)) + torch.sum(torch.abs(y))


## (h) Feature Extraction aus VGG-Schichten

Diese Funktion extrahiert **Content- und Style-Features** aus einem Eingabetensor mit Hilfe des VGG19-Modells. Dabei werden bestimmte Layer gezielt abgefragt, um die Repräsentationen für Inhalt und Stil zu erhalten.


### Was macht `get_features(model, tensor)`?

- Führt eine **Forward-Pass** durch das VGG-Modell aus
- Speichert die **Aktivierungen bestimmter Schichten** (jeweils als Feature-Maps)
- Verwendet:
  - **`relu4_2`** für den **Inhalt**
  - **Mehrere Schichten** (`relu1_2`, `relu2_2`, etc.) für den **Stil** (Gram-Matrizen)
- Berechnet Gram-Matrizen für Style-Features direkt im Funktionsverlauf


### Verwendete Layer im VGG19

| Layer-ID | Name im VGG      | Verwendung         |
|----------|------------------|--------------------|
| `"3"`    | `relu1_2`        | Stil-Feature       |
| `"8"`    | `relu2_2`        | Stil-Feature       |
| `"17"`   | `relu3_3`        | Stil-Feature       |
| `"26"`   | `relu4_3`        | Stil-Feature       |
| `"35"`   | `relu5_3`        | Stil-Feature       |
| `"22"`   | `relu4_2`        | Content-Feature    |

> Diese Layer wurden so gewählt, da sie sich im Originalpaper von Gatys et al. als besonders geeignet für die Darstellung von Stil- und Inhaltsinformationen erwiesen haben.


### Besonderheiten im Code

```python
if name == "22":
    features[layers[name]] = x
```
- Der Inhalt wird **direkt als Feature-Map gespeichert**

```python
features[layers[name]] = gram(x) / (H * W)
```
- Style-Features werden als **normierte Gram-Matrizen** gespeichert (nach Fläche)

```python
if name == "35":
    break
```
- Stoppt die Verarbeitung nach der letzten benötigten Schicht → spart Rechenzeit


> Diese extrahierten Features werden später für die Berechnung des Content- und Style-Loss verwendet.

In [ ]:
# -----------------------------------------------
# Feature Extraction aus VGG Layers
# -----------------------------------------------
def get_features(model, tensor):
    layers = {
        "3": "relu1_2",
        "8": "relu2_2",
        "17": "relu3_3",
        "26": "relu4_3",
        "35": "relu5_3",
        "22": "relu4_2",  # Content Layer
    }
    features = {}
    x = tensor
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            if name == "22":
                features[layers[name]] = x
            else:
                B, C, H, W = x.shape
                features[layers[name]] = gram(x) / (H * W)
        if name == "35":
            break
    return features


## (i) Startbild erzeugen

Bevor das stilisierte Bild optimiert werden kann, muss ein **Startbild** definiert werden. Dieses Bild ist der Ausgangspunkt für die iterative Optimierung, bei der sich das Bild schrittweise an Inhalt und Stil annähert.


### Funktion: `initial(content_tensor, init_image="random")`

Diese Funktion erzeugt das Startbild auf Basis des gewählten Initialisierungstyps.

#### Zwei Möglichkeiten der Initialisierung:

| Modus         | Beschreibung                                                                 |
|---------------|-------------------------------------------------------------------------------|
| `"random"`    | Erzeugt ein zufälliges Bild mit leichtem Rauschen                            |
| `"content"`   | Verwendet eine Kopie des Content-Bildes als Startbild                        |


### Was passiert im Code?

```python
if init_image == "random":
    tensor = torch.randn(C, H, W).mul(0.001).unsqueeze(0)
```
- Zufälliges Bild mit minimaler Varianz – verhindert zu starken Startbias  
- `.mul(0.001)` → sehr kleine Zufallswerte  
- `.unsqueeze(0)` → fügt Batch-Dimension `[1, C, H, W]` hinzu

```python
else:
    tensor = content_tensor.clone().detach()
```
- Wenn `"content"` gewählt ist, wird das Content-Bild als Basis genommen  
- `.clone().detach()` vermeidet unbeabsichtigte Verlinkung zum Original-Tensor


### Warum ist die Initialisierung wichtig?

- **Random** führt häufig zu interessanteren, aber unvorhersehbaren Ergebnissen
- **Content** sorgt für eine schnellere Konvergenz und stabilere Ergebnisse

> Die Wahl der Initialisierung beeinflusst also sowohl den Look als auch die Trainingsdynamik deines Style Transfers.

In [ ]:
# -----------------------------------------------
# Startbild erzeugen
# -----------------------------------------------
def initial(content_tensor, init_image="random"):
    B, C, H, W = content_tensor.shape
    if init_image == "random":
        tensor = torch.randn(C, H, W).mul(0.001).unsqueeze(0)
    else:
        tensor = content_tensor.clone().detach()
    return tensor


## (k) Stylization-Funktion: `stylize()`

Diese Funktion führt den eigentlichen **Style Transfer** durch. Dabei wird ein Bild so lange optimiert, bis es den Inhalt des Content-Bildes und den Stil des Style-Bildes kombiniert.

### Ablauf:

1. **Feature-Extraktion**:
   - Content-Features aus `relu4_2`
   - Style-Features aus mehreren VGG-Schichten (`relu1_2` bis `relu5_3`)

2. **Iterative Optimierung**:
   - In jeder Iteration wird das Bild `g` angepasst
   - Berechnet werden:
     - **Content Loss**
     - **Style Loss**
     - **TV Loss** (Glättung)

3. **Backpropagation**:
   - Der Gesamtverlust wird minimiert
   - Das Bild `g` wird direkt optimiert (nicht das Modell)

4. **Verlauf speichern**:
   - Alle Loss-Werte werden für spätere Visualisierung mitgeloggt

5. **Ausgabe und Visualisierung**:
   - Bild wird regelmäßig angezeigt und gespeichert
   - Optional mit Farbübertragung (`PRESERVE_COLOR`)

### Rückgabe:
- Das stilisierte Bild `g`
- Der komplette Verlustverlauf als Dictionary `loss_history`

In [ ]:
# -----------------------------------------------
# Stylization-Funktion
# -----------------------------------------------
def stylize(iteration=NUM_ITER):
    content_layers = ["relu4_2"]
    content_weights = {"relu4_2": 1.0}
    style_layers = ["relu1_2", "relu2_2", "relu3_3", "relu4_3", "relu5_3"]
    style_weights = {layer: 0.2 for layer in style_layers}

    c_feat = get_features(model, content_tensor)
    s_feat = get_features(model, style_tensor)

    # ===> Neu: Verlaufs-Listen für Plot
    loss_history = {"total": [], "content": [], "style": [], "tv": []}

    i = [0]
    while i[0] < iteration:

        def closure():
            optimizer.zero_grad()
            g_feat = get_features(model, g)

            c_loss = sum(
                content_weights[j] * content_loss(g_feat[j], c_feat[j])
                for j in content_layers
            )
            s_loss = sum(
                style_weights[j] * style_loss(g_feat[j], s_feat[j])
                for j in style_layers
            )

            c_loss *= CONTENT_WEIGHT
            s_loss *= STYLE_WEIGHT
            t_loss = TV_WEIGHT * tv_loss(g.clone().detach())
            total_loss = c_loss + s_loss + t_loss

            total_loss.backward(retain_graph=True)

            # ===> Verluste aufzeichnen
            loss_history["content"].append(c_loss.item())
            loss_history["style"].append(s_loss.item())
            loss_history["tv"].append(t_loss.item())
            loss_history["total"].append(total_loss.item())

            i[0] += 1
            if i[0] % SHOW_ITER == 1 or i[0] == NUM_ITER:
                print(
                    f"Iteration {i[0]}: Style {s_loss.item():.2f}, Content {c_loss.item():.2f}, TV {t_loss.item():.2f}, Total {total_loss.item():.2f}"
                )
                g_ = (
                    transfer_color(
                        ttoi(content_tensor.clone().detach()), ttoi(g.clone().detach())
                    )
                    if PRESERVE_COLOR == "True"
                    else ttoi(g.clone().detach())
                )
                show(g_)
                saveimg(g_, i[0] - 1)

            return total_loss

        optimizer.step(closure)

    return g, loss_history


## (l) Verlustverlauf visualisieren: `plot_losses()`

Diese Funktion visualisiert den Verlauf der verschiedenen Verlustkomponenten während des Style Transfers. So kannst du nachvollziehen, wie sich das Training entwickelt hat.

### Was wird geplottet?

- **Content Loss**
- **Style Loss**
- **Total Variation Loss (TV)**
- **Gesamtverlust (Total Loss)**

### Darstellung:

- Jeder Loss-Typ wird in einem eigenen **Subplot (2×2 Raster)** dargestellt
- Gemeinsame x-Achse: Iterationen
- Automatisches Styling mit `seaborn`

### Optional:
- Mit `save_path` kann der Plot als PNG gespeichert werden


In [ ]:
# -----------------------------------------------
# Interaktive Plots mit Plotly
# -----------------------------------------------
def plot_losses_interactive(
    losses,
    save_path="data/loss_plot_interactive.html",
    dark_mode=False,
    show_legend=False,
):
    """
    Erstellt einen interaktiven 2x2 Plot der Loss-Verläufe mit Plotly und speichert als HTML.

    Args:
        losses (dict): Dictionary mit Keys "total", "content", "style", "tv"
        save_path (str): Zielpfad zum Speichern
        dark_mode (bool): Ob Dark Mode verwendet werden soll
        show_legend (bool): Ob die Legende angezeigt wird
    """
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    df = pd.DataFrame(
        {
            "Iteration": list(range(1, len(losses["total"]) + 1)),
            "Total Loss": losses["total"],
            "Content Loss": losses["content"],
            "Style Loss": losses["style"],
            "TV Loss": losses["tv"],
        }
    )

    fig = sp.make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("Total Loss", "Content Loss", "Style Loss", "TV Loss"),
        horizontal_spacing=0.12,
        vertical_spacing=0.15,
    )

    # Farben + Setup
    colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
    keys = ["Total Loss", "Content Loss", "Style Loss", "TV Loss"]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    for key, color, pos in zip(keys, colors, positions):
        fig.add_trace(
            go.Scatter(
                x=df["Iteration"],
                y=df[key],
                mode="lines",
                name=key,
                line=dict(color=color, width=2),
                hovertemplate=f"<b>{key}</b><br>Iteration: %{{x}}<br>Loss: %{{y:.2e}}<extra></extra>",
            ),
            row=pos[0],
            col=pos[1],
        )

    # Layout
    fig.update_layout(
        height=800,
        width=1200,
        title_text="📈 <b>Interaktiver Loss-Verlauf pro Komponente</b>",
        title_x=0.5,
        showlegend=show_legend,
        template="plotly_dark" if dark_mode else "plotly_white",
        font=dict(size=14),
        margin=dict(t=80, b=40),
    )

    # Achsentitel einheitlich
    for i in range(1, 5):
        fig["layout"][f"yaxis{i}"].title = "Loss"
        fig["layout"][f"xaxis{i}"].title = "Iteration"

    fig.write_html(save_path)
    print(f"✅ Interaktiver Plot gespeichert unter: {save_path}")
    fig.show()


## (m) Ausführung: Style Transfer starten

In diesem Abschnitt wird der vollständige Style Transfer ausgeführt.

### Schritte:

1. **Bild-zu-Tensor-Konvertierung**
   ```python
   content_tensor = itot(content_img).to(device)
   style_tensor = itot(style_img).to(device)
   ```
   - Die geladenen Bilder werden für das Modell vorbereitet

2. **Startbild erzeugen**
   ```python
   g = initial(content_tensor, init_image=INIT_IMAGE).to(device).requires_grad_(True)
   ```
   - Entweder zufälliges Rauschen oder das Content-Bild als Ausgangspunkt

3. **Optimierer festlegen**
   ```python
   optimizer = optim.Adam([g], lr=ADAM_LR)
   ```
   - Wahl zwischen `adam` (standardmäßig) oder `lbfgs`

4. **Style Transfer starten**
   ```python
   out, losses = stylize(iteration=NUM_ITER)
   ```
   - Das Bild `g` wird über mehrere Iterationen so angepasst, dass es Stil und Inhalt kombiniert


> Nach Abschluss enthält `out` das finale stilisierte Bild und `losses` die Entwicklung der Verlustfunktionen.

In [ ]:
# -----------------------------------------------
# Ausfuehrung
# -----------------------------------------------
content_tensor = itot(content_img).to(device)
style_tensor = itot(style_img).to(device)
g = initial(content_tensor, init_image=INIT_IMAGE).to(device).requires_grad_(True)

if OPTIMIZER == "lbfgs":
    optimizer = optim.LBFGS([g])
elif OPTIMIZER == "adam":
    optimizer = optim.Adam([g], lr=ADAM_LR)

# Stylize!
out, losses = stylize(iteration=NUM_ITER)


## (n) Loss-Verlauf anzeigen

Nach dem Style Transfer wird der Verlauf der einzelnen Verlustkomponenten (Content, Style, TV, Total) visuell dargestellt.

```python
plot_losses(losses)
```

### Ziel:
- **Verständnis für den Optimierungsverlauf** entwickeln
- **Überwachung** der Konvergenz und Stabilität
- **Identifikation von Ungleichgewichten** in der Gewichtung der Loss-Funktionen

> Die Visualisierung hilft dir, das Verhalten des Modells über die Trainingszeit zu interpretieren und ggf. Hyperparameter anzupassen.

In [ ]:
plot_losses_interactive(losses, dark_mode=True, show_legend=True)


## Zusammenfassung: Was passiert hier?

Dieser Code implementiert den klassischen **Neural Style Transfer** (nach Gatys et al., 2015).

### Was wird gemacht?

- **VGG19** wird als Feature-Extractor verwendet (nicht trainiert!)
- Das **generierte Bild wird direkt optimiert**
- Verluste:
  - **Content Loss** (zwischen Feature-Maps)
  - **Style Loss** (über Gram-Matrizen)
  - **TV Loss** (Bildglättung)
- Bild wird iterativ angepasst (nicht das Modell!)
- Ergebnis: Ein stilisiertes Bild + Loss-Verlauf


## Wie trainiert man stattdessen ein eigenes Modell?

Beim **Fast Style Transfer** wird ein **CNN-Modell** trainiert, das beliebige Bilder im gewünschten Stil in **Echtzeit** umwandeln kann.

### Dafür braucht man:

1. **Datensatz** mit vielen Content-Bildern
2. **Feedforward-Stilnetz** (z. B. ResNet oder Encoder–Decoder)
3. **VGG19 (eingefroren)** für die Loss-Berechnung
4. **Loss-Funktionen**: Content + Style + TV Loss
5. **Training-Loop**: Optimierung des Netzwerks, nicht des Bildes


### Vergleich

| Klassisch (hier)        | Fast Style Transfer         |
|-------------------------|-----------------------------|
| Optimiert Bild direkt   | Trainiert CNN-Modell        |
| Flexibel, aber langsam  | Schnell, aber stilgebunden  |
| Kein Training nötig     | Modelltraining erforderlich |

